**Install and Import Libraries**

This section installs and imports the necessary libraries for building the image classification model, including TensorFlow for building and training the model, and other libraries for data manipulation and visualization.

In [26]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

**Load and Preprocess Data**

This section defines utility functions to handle image data. It includes a function to check for corrupted images to ensure data quality. The main part loads the images from the specified directory, extracts labels from filenames, and filters out any corrupted images before preparing them for the model.

In [27]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

def is_image_corrupted(filepath):
    try:
        with Image.open(filepath) as img:
            img.verify()
        return False
    except (IOError, SyntaxError) as e:
        print(f'Corrupted image: {filepath}, error: {e}')
        return True

In [28]:
def load_data(data_dir):
    images = []
    labels = []
    
    for filename in os.listdir(data_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(data_dir, filename)
            if not is_image_corrupted(filepath):
                label = filename.split('-')[0]
                images.append(filepath)
                labels.append(label)
    
    return images, labels

image_dir = 'images/'
images, labels = load_data(image_dir)

print(f'Loaded {len(images)} images.')

Loaded 300 images.


**Split Data and Create Data Generators**

The dataset is split into training and validation sets to evaluate the model's performance on unseen data. `ImageDataGenerator` is used to preprocess the images and apply data augmentation to the training set, which helps in preventing overfitting and improving model generalization.

In [29]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

import pandas as pd
train_df = pd.DataFrame({'filename': X_train, 'class': y_train})
val_df = pd.DataFrame({'filename': X_val, 'class': y_val})

img_size = (224, 224)
batch_size = 32

In [30]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='filename',
    y_col='class',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 240 validated image filenames belonging to 2 classes.
Found 60 validated image filenames belonging to 2 classes.


**Build the Model**

A pre-trained model, MobileNetV2, is used as the base for our classifier. The top layers of the pre-trained model are frozen, and new custom layers are added for our specific classification task. This approach, known as transfer learning, leverages the powerful features learned by the base model on a large dataset, leading to faster and more accurate training on our smaller dataset.

In [31]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

AttributeError: 'DataFrameIterator' object has no attribute 'num_classes'

**Compile and Train the Model**

The model is compiled with the Adam optimizer and categorical cross-entropy loss function. It is then trained for a few epochs on the training data, and its performance is monitored on the validation set.

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

**Evaluate the Model**

The model's performance is evaluated by plotting the training and validation accuracy and loss over epochs. This helps in visualizing how well the model is learning and generalizing.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

**Confusion Matrix**

A confusion matrix is generated to provide a detailed breakdown of the model's classification performance for each class. It shows the number of correct and incorrect predictions, which helps in understanding the model's strengths and weaknesses.

In [ ]:
Y_pred = model.predict(val_generator)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(val_generator.classes, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=val_generator.class_indices.keys(), yticklabels=val_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print('Classification Report')
print(classification_report(val_generator.classes, y_pred, target_names=val_generator.class_indices.keys()))

**Predict on a Sample Image**

This section provides a function to predict the class of a single image. It loads an image, preprocesses it to match the model's input requirements, and then uses the trained model to make a prediction. The predicted class with the highest probability is then displayed.

In [ ]:
def predict_image(filepath, model, class_indices):
    img = Image.open(filepath)
    img = img.resize((224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)

    class_labels = {v: k for k, v in class_indices.items()}
    return class_labels[predicted_class[0]]

sample_image = 'images/greater-flamingo-1.jpeg'
predicted_label = predict_image(sample_image, model, train_generator.class_indices)
print(f'The predicted label for the image is: {predicted_label}')

**Save the Model**

The trained model, including its architecture and weights, is saved to a file. This allows for easy reuse in the future without needing to retrain. The class mappings are also saved to a JSON file for later reference.

In [ ]:
model.save('artifacts/visual_classifier_2.keras')

import json
class_indices = train_generator.class_indices
class_mapping = {v: k for k, v in class_indices.items()}
with open('artifacts/class_mapping_2.json', 'w') as f:
    json.dump(class_mapping, f)